In [1]:
from difflogic import LogicLayer, GroupSum
import torch
from chop import MaseGraph

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import os
# import sys

# "Add dot (pydot) to path if needed:"

# new_path = os.path.expanduser("~/miniforge3/envs/adls-project/bin/")
# if new_path not in sys.path:
#     sys.path.append(new_path)

# # Add to environment PATH as well
# os.environ["PATH"] = new_path + os.pathsep + os.environ["PATH"]

In [3]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    LogicLayer(4, 4),
    GroupSum(k=2, tau=20)
)

In [4]:
CUSTOM_OPS = {
    "modules": {
        LogicLayer: {
            "args": {
                "input": "data_in"
            },
            "toolchain": "INTERNAL_RTL",
            "module": "fixed_difflogic_logic",
            "dependence_files": [
                "difflogic_layers/rtl/fixed_difflogic_logic.sv",
                "difflogic_layers/rtl/fixed_difflogic_logic_neuron.sv"
            ],
        },
        GroupSum: {
            "args": {
                "input": "data_in"
            },
            "toolchain": "INTERNAL_RTL",
            "module": "difflogic_groupsum",
            "dependence_files": [],
        },
    },
    "functions": {},
}

mg = MaseGraph(model, custom_ops=CUSTOM_OPS)

# mg.draw("DLG.svg")

In [5]:
def eval(model, loader, mode):
    orig_mode = model.training
    with torch.no_grad():
        model.train(mode=mode)
        res = np.mean(
            [
                (model(x.to('cpu').round()).argmax(-1) == y.to('cpu')).to(torch.float32).mean().item()
                for x, y in loader
            ]
        )
        model.train(mode=orig_mode)
    return res.item()

In [6]:
import mnist_dataset
import numpy as np

train_set = mnist_dataset.MNIST('./data-mnist', train=True, download=True, remove_border=True)
test_set = mnist_dataset.MNIST('./data-mnist', train=False, remove_border=True)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True, pin_memory=True, drop_last=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False, pin_memory=True, drop_last=True)

state_dict = torch.load("best_model_full.pth", map_location=torch.device('cpu'))  # Load the dictionary
# mg.model.load_state_dict(state_dict, strict=False)  # Apply the weights

# mg.model.eval()

# print("Accuracy: ", eval(mg.model, test_loader, mode=False))

In [7]:
import types

def test_pass(
    graph,
    pass_args={"model":"None", "state_dict":{}},
):
#     model = pass_args["model"]
#     given_params = model.state_dict()
#     if (pass_args["state_dict"]):
#         given_params = pass_args["state_dict"]
    
    
    # Step 1: Copy input placeholder nodes
    for node in graph.nodes:
        print(node.op)
        print(node.meta["mase"].parameters)
        print()
#         if "LogicLayer" in node.op:  # Detect input nodes
            
    

    return mg, None
                    
                    
                    
#     nodes = set({})
#     for node in mg.fx_graph.nodes:
#         if (node.op == "call_module"):
#             pass
#         elif (node.op == "call_method" and long not in node.target ):
#             if len(node.args) > 1:
#                 raise "What?"
#             node.replace_all_uses_with(node.args[0])
#         elif (isinstance(node.target, types.BuiltinFunctionType):
#              node.replace_all_uses_with(node.args[0]) 
            
            
#     return graph, None

In [8]:
import chop.passes as passes

dummy_in_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True, drop_last=True)
image, label = next(iter(dummy_in_loader))
print(image)
image = torch.randn((1,2,2))
# image = image.cpu().squeeze(0) 

mg_test, _ = passes.init_metadata_analysis_pass(mg)
mg_test, _ = passes.add_common_metadata_analysis_pass(
    mg_test,
    pass_args={
        "dummy_in": {"input_1": image}, #input name Hardcoded here, change
#         "add_value": False,
    },
)

# There is error below, need to fix
# mg_test, _ = passes.add_hardware_metadata_analysis_pass(mg_test)


mg_test.draw("DLG_test.svg")


# below pass just prints out the metadata for us to debug
# test_pass(mg)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


tensor([[[[0.0000, 0.0000, 0.3294, 0.7255, 0.6235, 0.5922, 0.2353, 0.1412,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.8706, 0.9961, 0.9961, 0.9961, 0.9961, 0.9451,
           0.7765, 0.7765, 0.7765, 0.7765, 0.7765, 0.7765, 0.7765, 0.7765,
           0.6667, 0.2039, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.2627, 0.4471, 0.2824, 0.4471, 0.6392, 0.8902,
           0.9961, 0.8824, 0.9961, 0.9961, 0.9961, 0.9804, 0.8980, 0.9961,
           0.9961, 0.5490, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0667,
           0.2588, 0.0549, 0.2627, 0.2627, 0.2627, 0.2314, 0.0824, 0.9255,
           0.9961, 0.4157, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3255, 0.9922,
           0.8196, 0.0706, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0

# Hardware Stuff!

In [9]:
for node in mg_test.nodes:
    print(node.meta)

{'mase': <chop.ir.graph.mase_metadata.MaseMetadata object at 0x7fff5b920c10>}
{'nn_module_stack': OrderedDict([('0', ('0', <class 'torch.nn.modules.flatten.Flatten'>))]), 'mase': <chop.ir.graph.mase_metadata.MaseMetadata object at 0x7fff5ed74d50>}
{'nn_module_stack': OrderedDict([('1', ('1', <class 'difflogic.difflogic.LogicLayer'>))]), 'mase': <chop.ir.graph.mase_metadata.MaseMetadata object at 0x7ffff81a3d90>}
{'nn_module_stack': OrderedDict([('2', ('2', <class 'difflogic.difflogic.GroupSum'>))]), 'mase': <chop.ir.graph.mase_metadata.MaseMetadata object at 0x7fff5b94d410>}
{'mase': <chop.ir.graph.mase_metadata.MaseMetadata object at 0x7fff5b94e5d0>}


In [10]:
from chop.passes.graph.analysis import add_hardware_metadata_analysis_pass
mg_test, _ = add_hardware_metadata_analysis_pass(mg_test)

DEBUG:  None
DEBUG:  difflogic_groupsum


In [11]:
def print_hardware_metadata(graph, args={}):
    for node in graph.nodes:
        print(node.op)
        for k, v in node.meta["mase"]["hardware"].items():
            print(f"{k}: {v}")
        print()
    return mg, None

In [12]:
from chop.passes.graph.transforms import emit_internal_rtl_transform_pass

mg_test, _ = emit_internal_rtl_transform_pass(mg_test)

In [13]:
mg_test, _ = print_hardware_metadata(mg_test)

placeholder
is_implicit: True
device_id: 0
max_parallelism: [4, 4, 4, 4]

call_module
is_implicit: False
device_id: -1
interface: {}
toolchain: INTERNAL_HLS
module: None
dependence_files: []
max_parallelism: [4, 4, 4, 4]
verilog_param: {'DATA_IN_0_PRECISION_0': 32, 'DATA_IN_0_TENSOR_SIZE_DIM_0': 2, 'DATA_IN_0_PARALLELISM_DIM_0': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_1': 2, 'DATA_IN_0_PARALLELISM_DIM_1': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_2': 1, 'DATA_IN_0_PARALLELISM_DIM_2': 1, 'DATA_OUT_0_PRECISION_0': 32, 'DATA_OUT_0_TENSOR_SIZE_DIM_0': 4, 'DATA_OUT_0_PARALLELISM_DIM_0': 4, 'DATA_OUT_0_TENSOR_SIZE_DIM_1': 1, 'DATA_OUT_0_PARALLELISM_DIM_1': 1}

call_module
is_implicit: False
device_id: -1
interface: {}
toolchain: INTERNAL_RTL
module: fixed_difflogic_logic
dependence_files: ['difflogic_layers/rtl/fixed_difflogic_logic.sv', 'difflogic_layers/rtl/fixed_difflogic_logic_neuron.sv']
max_parallelism: [4, 4, 4, 4]
verilog_param: {'DATA_IN_0_TENSOR_SIZE_DIM_0': 4, 'DATA_OUT_0_TENSOR_SIZE_DIM_0': 4, 'LAYER_

In [14]:
from chop.passes.graph.transforms import emit_bram_transform_pass

mg_test, _ = emit_bram_transform_pass(mg_test)

In [15]:
from chop.passes.graph.transforms import emit_verilog_top_transform_pass

mg_test, _ = emit_verilog_top_transform_pass(mg_test)